##Importing Required Standard Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score,roc_curve,f1_score
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [4]:
main_train_df = pd.read_csv("/content/drive/MyDrive/ML/ML_Project/datasets/train_data.csv")
main_test_df = pd.read_csv("/content/drive/MyDrive/ML/ML_Project/datasets/test_data.csv")

In [5]:
working_df = main_train_df.copy()

In [6]:
working_df.head(10)

,url,webpageDescription,alchemy_category,alchemy_category_score,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,embedRatio,framebased,frameTagRatio,domainLink,tagRatio,imageTagRatio,isNews,lengthyDomain,hyperlinkToAllWordsRatio,isFrontPageNews,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
0,http://www.polyvore.com/cgi/home?id=1389651,"{""title"":""Join Polyvore and follow the latest ...",?,?,1.916667,0.047619,0.007937,0.000000,0.000000,0.803797,0.0,0,0.027778,0,0.203161,3.120000,?,0,34,0,682,126,1,0.531746,0.142857,1,3711
1,http://www.youtube.com/watch?v=ippMPPu6gh4,"{""body"":""Speed Air Man--David Belle david bell...",?,?,1.257576,0.141026,0.000000,0.000000,0.000000,1.142857,0.1,0,0.015086,0,0.168189,13.100000,?,0,12,0,3008,78,1,0.628205,0.000000,1,7222
2,http://www.musingsofahousewife.com/2011/03/tri...,"{""title"":""Chicken Gruyere Recipe "",""body"":""Chi...",science_technology,0.386685,2.024000,0.630350,0.284047,0.233463,0.202335,0.443409,0.0,0,0.033935,0,0.192652,0.126582,1,1,17,0,11008,257,5,0.466926,0.062500,1,3964
3,http://www.thelittleteochew.com/2011/07/ikan-b...,"{""title"":""Ikan Bilis With Tempeh Groundnuts "",...",recreation,0.475039,1.665254,0.419580,0.199301,0.097902,0.066434,0.472649,0.0,0,0.036530,0,0.198000,0.135593,1,1,28,0,4585,286,5,0.244755,0.085868,1,3697
4,http://recipes.wuzzle.org/index.php/72,"{""url"":""recipes wuzzle org index php 72"",""titl...",computer_internet,0.535009,0.181818,0.036364,0.000000,0.000000,0.000000,0.292614,0.0,0,0.015152,0,0.241564,0.054217,?,0,3,0,1745,55,1,0.072727,0.115044,1,4321
5,http://www.sleepdisordersguide.com/blog/good-n...,"{""title"":""Discover The Threats Of Bad Sleeping...",health,0.953935,3.697917,0.848485,0.636364,0.191919,0.101010,0.382585,0.0,0,0.022449,0,0.254480,0.003398,1,1,23,0,6061,99,16,0.010101,0.105439,1,6940
6,http://www.insidershealth.com/natural_cure/poi...,"{""title"":""InsidersHealth com insidershealth.co...",health,0.893828,1.691781,0.469799,0.147651,0.026846,0.006711,0.522134,0.0,0,0.026190,0,0.264697,0.045455,?,0,57,0,961,149,4,0.006711,0.086957,1,5416
7,http://yawoot.com/post/2122,"{""title"":""Well the world is made of peanut but...",recreation,0.550131,1.170213,0.634921,0.000000,0.000000,0.000000,0.224906,0.0,0,0.040248,0,0.181396,0.011561,?,0,11,0,1865,63,1,0.031746,0.123209,0,7349
8,http://www.bbc.co.uk/news/technology-22516169,"{""url"":""bbc co news technology 22516169"",""titl...",computer_internet,0.537425,3.466667,0.453704,0.194444,0.138889,0.129630,0.604119,0.0,0,0.111582,0,0.234397,0.708861,1,1,64,0,977,108,2,0.064815,0.085106,0,6727
9,http://fxcuisine.com/default.asp?Display=163&r...,"{""title"":""Sicilian Tangerine Sorbet "",""body"":""...",computer_internet,0.287329,0.941176,0.117647,0.039216,0.000000,0.000000,0.487411,0.0,0,0.088785,0,0.303562,0.034483,1,0,7,0,2625,51,5,0.705882,0.094643,1,5155


In [7]:
working_df.describe()

,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,embedRatio,framebased,frameTagRatio,domainLink,tagRatio,imageTagRatio,lengthyDomain,hyperlinkToAllWordsRatio,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
count,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.0,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000,5916.000000
mean,2.852545,0.469274,0.214425,0.092066,0.049570,2.298490,-0.106884,0.0,0.056298,0.019270,0.233858,0.255667,0.660412,30.086207,5704.955544,179.137086,4.954699,0.170699,0.100854,0.517241,3699.003888
std,9.559954,0.203990,0.147265,0.096179,0.073347,5.771091,0.310639,0.0,0.041516,0.137483,0.053073,1.438010,0.473609,20.500121,8814.892098,184.745134,3.229823,0.179872,0.077778,0.499745,2132.447652
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,0.000000,0.000000,0.045564,-1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,1.600000,0.342038,0.105682,0.022222,0.000000,0.442573,0.000000,0.0,0.028571,0.000000,0.200965,0.025726,0.000000,14.000000,1574.750000,82.000000,3.000000,0.040984,0.068235,0.000000,1860.750000
50%,2.086957,0.484084,0.202532,0.068909,0.022690,0.483416,0.000000,0.0,0.045606,0.000000,0.230278,0.083622,1.000000,25.000000,3484.000000,139.000000,5.000000,0.113347,0.089238,1.000000,3703.500000
75%,2.622778,0.616681,0.300000,0.132231,0.065990,0.577449,0.000000,0.0,0.073227,0.000000,0.260889,0.242456,1.000000,43.000000,6316.000000,222.000000,7.000000,0.238395,0.112486,1.000000,5539.250000
max,363.000000,1.000000,0.982906,0.980392,0.980392,21.000000,0.250000,0.0,0.444444,1.000000,0.716883,52.000000,1.000000,100.000000,207952.000000,4997.000000,22.000000,1.000000,1.000000,1.000000,7394.000000


##Language Preprocessing

Extracting Body From json text


In [8]:
import json

In [9]:
web_body_list = []
web_title_list = []
web_url_list = []
for i in range(working_df.shape[0]):
  dictionary_json =   json.loads(working_df['webpageDescription'][i])
  if('body' in dictionary_json):
    if(dictionary_json['body'] is None):
      web_body_list.append(" ")
    else:
      web_body_list.append(dictionary_json['body'])
  else:
    web_body_list.append(" ")

  if('title' in dictionary_json):
    if(dictionary_json['title'] is None):
      web_title_list.append(" ")
    else:
      web_title_list.append(dictionary_json['title'])
    
  else:
    web_title_list.append(" ")

  if('url' in dictionary_json):
    if(dictionary_json['url'] is None):
      web_url_list.append(" ")
    else:
      web_url_list.append(dictionary_json['url'])
  else:
    web_url_list.append(" ")
   

In [10]:
web_text_list = []
for i in range(working_df.shape[0]):
  web_text = web_body_list[i] + " " + web_title_list[i] + " " + web_url_list[i]
  web_text_list.append(web_text)

##Natural Language Processing Techniques on Body Text

TF-IDF Approach

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tv = TfidfVectorizer()

In [13]:
nlp_X = tv.fit_transform(web_text_list).toarray()

In [14]:
nlp_X.shape

(5916, 80032)

##Splitting the Data

In [15]:
train_Y = working_df["label"].to_numpy().astype(np.float64)

##Modeling

In [ ]:
# scores = cross_val_score(MultinomialNB(alpha = 0.45),nlp_X,train_Y, scoring = 'roc_auc')
# print(scores)
# print(np.mean(scores))

LogisticRegression

In [17]:
log_reg_model = LogisticRegression(solver = 'liblinear')

In [18]:
log_reg_model.fit(nlp_X,train_Y)

LogisticRegression(solver='liblinear')

In [19]:
pred_log_reg =log_reg_model.predict_proba(nlp_X).T[1]

In [20]:
roc_auc_score(train_Y,pred_log_reg)

0.9468211631057653

Multinomial Naive Bayes

In [21]:
multinomialNB_model = MultinomialNB(alpha = 0.45)

In [22]:
multinomialNB_model.fit(nlp_X,train_Y)

MultinomialNB(alpha=0.45)

In [23]:
pred_multiNB = multinomialNB_model.predict_proba(nlp_X).T[1]

In [24]:
roc_auc_score(train_Y,pred_multiNB)

0.9324170190952199

##Testing

Preprocessing on Test Set

###NLP on Test Set

In [25]:
test_web_body_list = []
test_web_title_list = []
test_web_url_list = []
for i in range(main_test_df.shape[0]):
  dictionary_json =   json.loads(main_test_df['webpageDescription'][i])
  if('body' in dictionary_json):
    if(dictionary_json['body'] is None):
      test_web_body_list.append(" ")
    else:
      test_web_body_list.append(dictionary_json['body'])
  else:
    test_web_body_list.append(" ")

  if('title' in dictionary_json):
    if(dictionary_json['title'] is None):
      test_web_title_list.append(" ")
    else:
      test_web_title_list.append(dictionary_json['title'])
    
  else:
   test_web_title_list.append(" ")

  if('url' in dictionary_json):
    if(dictionary_json['url'] is None):
      test_web_url_list.append(" ")
    else:
      test_web_url_list.append(dictionary_json['url'])
  else:
    test_web_url_list.append(" ")

In [26]:
test_web_text_list = []
for i in range(main_test_df.shape[0]):
  test_web_text = test_web_body_list[i] + " " + test_web_title_list[i] + " " + test_web_url_list[i]
  test_web_text_list.append(test_web_text)

In [27]:
nlp_test_X = tv.transform(test_web_text_list).toarray()

In [52]:
final_Ytest = (2*log_reg_model.predict_proba(nlp_test_X).T[1] + multinomialNB_model.predict_proba(nlp_test_X).T[1])/3

In [53]:
final_Ytest.sum()

772.6179318406614

##Preparation For Submission

In [37]:
submission = pd.read_csv("/content/drive/MyDrive/ML/ML_Project/datasets/sample_submission.csv")

In [38]:
submission.head()

,id,label
0,7172,0.597802
1,2687,0.177493
2,6990,0.619266
3,5639,0.868449
4,2701,0.341868


In [48]:
submission["label"] = final_Ytest

In [40]:
submission.head()

,id,label
0,7172,0.368879
1,2687,0.860622
2,6990,0.969373
3,5639,0.511303
4,2701,0.283147


In [49]:
submission.to_csv("/content/drive/MyDrive/ML/ML_Project/Submissions/submission1.csv",index = False)